In [17]:
from sklearn import datasets
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import roc_auc_score
from scipy.spatial.distance import euclidean

In [9]:
df = pd.read_csv("data-logistic.csv", names=["target", "w1", "w2"])

In [53]:
y = df.target
X = df.drop("target", axis=1)

-0.66382653682499992

In [57]:
def step(col, name, W, k, X, y, C):
    l = X.count()[0]
    sum = 0
    for i in range(l):
       sum += y[i] * X[name][i] * (1 - 1 / (1 + math.exp(-y[i] * (W[0] * X.w1[i] + W[1] * X.w2[i]))))    
    main = k  / l * sum
    regularization = k * C * W[col]
    return W[col] + main - regularization

In [93]:
def learn(C, y, X, k = 0.1, w = pd.Series([0, 0])):
    max_iter = 10000
    epsilon = 1e-5
    index = 0
    w_last = pd.Series([1000, 10000])
    l = X.count()[0]
    while index < max_iter and euclidean(w, w_last) > epsilon:
        w_last = w
        w1 = step(0, "w1", w, k, X, y, C)
        w2 = step(1, "w2", w, k, X, y, C)
        w = pd.Series([w1, w2])
        index += 1
    print(f"Total steps: {index}, diff: {euclidean(w, w_last)}")
    return w

In [67]:
w_without_reg = learn(0, y, X)
w_without_reg

Total steps: 244, diff: 9.802349781627123e-06


0    0.287812
1    0.091983
dtype: float64

In [94]:
w_with_reg = learn(10, y, X)
w_with_reg

Total steps: 8, diff: 4.763189139558198e-06


0    0.028559
1    0.024780
dtype: float64

In [77]:
#def  a(W x) = 1 / (1 + exp(-w1 x1 - w2 x2))
#a_without_reg = 
def a(w):
    return X.apply(lambda x: 1 / (1 + math.exp(-x.w1 * w[0]  - x.w2 * w[1])), axis=1)

In [79]:
a_wihtout_reg = a(w_without_reg)
a_with_reg = a(w_with_reg)

In [81]:
roc_auc_score(y, a_wihtout_reg)

0.92685714285714282

In [82]:
roc_auc_score(y, a_with_reg)

0.93628571428571417

In [83]:
with open("log_regression_res.txt", "w") as f:
    print("%.3f %.3f" % (roc_auc_score(y, a_wihtout_reg), roc_auc_score(y, a_with_reg)), file=f, end='')

In [100]:
learn(10, y, X, 0.1, pd.Series([20, 20]))

Total steps: 10, diff: 3.1039501171097285e-06


0    0.028560
1    0.024781
dtype: float64